In [ ]:
import numpy as np
import cartopy.crs as ccrs
import colorcet as cc

import holoviews as hv
import geoviews as gv

from vis_tools import read_hysplit_netcdf

from bokeh.models.glyphs import Image
from bokeh.models.renderers import GlyphRenderer
from bokeh.models.formatters import FuncTickFormatter
from bokeh.models.annotations import ColorBar, Legend
from bokeh.tile_providers import STAMEN_TERRAIN_RETINA

# squish a bunch of benign Matplotlib warnings related to NaN's
import warnings
warnings.filterwarnings('ignore', message='Warning: converting a masked element to nan.')
warnings.filterwarnings('ignore', message='invalid value encountered in greater')
warnings.filterwarnings('ignore', message='invalid value encountered in less')
warnings.filterwarnings('ignore', message='No contour levels were found within the data range.')

hv.extension('bokeh')
renderer = hv.renderer('bokeh')

###############################################
# SPECIFY: file name and path for HYSPLIT model
FILENAME = '18042918_taupo_15.0_0.01.nc'

# SPECIFY:
ASH_MIN = 10**-1  # min ash colorbar cutoff
ASH_MAX = 10**2  # max ash colorbar cutoff
###############################################
       
model = read_hysplit_netcdf(FILENAME, ASH_MIN)
model['total_deposition'].values = np.log10(model['total_deposition'].values)  # manually take the log

volc_loc = (model.attrs['volcano_location'][1], model.attrs['volcano_location'][0])

gv_ds = gv.Dataset(model, crs=ccrs.PlateCarree())
gv_ds = gv_ds.redim.range(total_deposition=(np.log10(ASH_MIN), np.log10(ASH_MAX)))  # clip colorbar
gv_ds = gv_ds.redim.label(time='Time')

contour_levels = np.arange(np.log10(ASH_MIN)+1, np.log10(ASH_MAX)+1)  # log-spaced contours (skip ASH_MIN contour)

background_map = gv.WMTS(STAMEN_TERRAIN_RETINA)
model_image = gv_ds.to(gv.Image, ['lon', 'lat'])
model_contours = gv.operation.contours(model_image, levels=contour_levels)
source_location = gv.Points(volc_loc, crs=ccrs.Geodetic(), label='Source')

fig = background_map * model_image * model_contours * source_location

# hook function for Bokeh plot adjustments
def adjust_plot(plot, element):
    p = plot.handles['plot']
            
    # modify tools
    tools = dict(zip(map(lambda tool: str(tool).split('(')[0], p.tools), p.tools))
    wz = tools['WheelZoomTool']
    wz.zoom_on_axis = False
    p.tools = [tools['PanTool'], tools['BoxZoomTool'], wz, tools['ResetTool']]
    p.toolbar.active_scroll = wz
    
    # modify plot elements
    for object in p.renderers:    
        if isinstance(object, GlyphRenderer):
            if isinstance(object.glyph, Image):
                object.glyph.global_alpha = 0.5  # set the global alpha for Bokeh Images  
        elif isinstance(object, ColorBar):
            object.ticker.desired_num_ticks = len(contour_levels)+1  # set the number of colorbar ticks
        elif isinstance(object, Legend):
            object.click_policy = 'none'
            i = 0
            while list(object.items[i].label.keys())[0] != 'value':
                i = i + 1
            object.items = [object.items[i]]  # remove contours from the legend            
                                          
cmap=cc.b_linear_kry_5_98_c75[::-1]    
plot_opts = {'Image': {'style': dict(cmap=cmap),
                        'plot': dict(colorbar=True, title_format=FILENAME.split('/')[-1], height=600, width=700,
                                     colorbar_opts=dict(location=(0, 0), orientation='horizontal',
                                                        title='Ash thickness (mm)',
                                                        formatter=FuncTickFormatter(code='''return(Math.pow(10, tick));''')),
                                     colorbar_position='bottom', finalize_hooks=[adjust_plot])},
          'Contours': {'style': dict(cmap=cmap, line_width=1.5),
                        'plot': dict()},
            'Points': {'style': dict(size=20, marker='triangle', line_color='black', fill_color='cyan'),
                        'plot': dict()}
            }             
fig = fig.opts(plot_opts)

fig
#renderer.save(fig, 'geoviews_bokeh_map'); print('HTML file saved')